### 以下代码用来实现经典的编码器－解码器代码，并使用 cmn_eng　2.2w条翻译数据，作为实例

In [2]:
import tensorflow as tf
import os

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

In [3]:
print(tf.__version__)

2.0.0


In [4]:
import unicodedata
import re
import numpy as np
import os
import io
import time